# Function Calling - How it work

Function Calling でのReasoning Engineの挙動を確認するコードです。

参考:
https://learn.microsoft.com/ja-jp/azure/ai-services/openai/how-to/function-calling

## シナリオ
出張申請を行うため、幾つかの関数呼び出しを行います。
- 出張申請規定のドキュメントの参照。ここがポイントで、出張申請規定が変わると、それぞれ呼び出すAPIのパラメーターが連動して変化します。
    - [国内出張申請規定](./docs/domestic_travel_policy.txt)
- スケジュール確認
- (まだ呼び出していない)出張申請のメール作成
- 航空機の手配
- ホテルの手配

## 環境作成
- JupyterNotebook
- 必要なPythonのパッケージは、`requirements.txt`に記載しています。
    - 以下のコマンドでインストールしてください。
    ```
    pip install -f requirements.txt
    ```
- Azure OpenAI Service のインスタンスを作成してください。
    - 作成方法は、[こちら](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python#retrieve-key-and-endpoint)を参照してください。
    - 作成したインスタンスのエンドポイントとキーとデプロイしたモデルの名称を取得してください。
    - このリポジトリーの直下に`.env`ファイルを作成して、そこに記載して、環境変数として設定します。
    ```

## 注意
- 実装している関数は、全てスタブ扱いです。実際のREST API呼び出しなどは、適時実装する必要があります。

## 環境変数の読み込み


In [1]:
%load_ext dotenv
%dotenv

# Azure OpenAI Service の利用設定

In [2]:
import os
import json
from openai import AzureOpenAI
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview"
)

max_response_tokens = 1024

deployment_name = os.getenv("AZURE_OPENAI_MODEL_NAME")


# [関数] 来週のスケジュールの返信

In [3]:
# ``` GitHub Copilot のPrompt
# スケジュール管理の関数を作成します。サンプルとして、来週の木曜日の13:00から17:00まで、福岡。同じく来週の金曜日の9:00-12:00まで福岡の予定を返却します。

def get_schedule():
    """Get the schedule for the next week"""
    print("get_schedule called")  
    next_thursday = datetime.now() + timedelta(days=(3 - datetime.now().weekday()) % 7)
    next_friday = datetime.now() + timedelta(days=(4 - datetime.now().weekday()) % 7)
    
    return json.dumps({
        "schedule": [
            {
                "date": next_thursday.strftime("%Y-%m-%d"),
                "location": "fukuoka",
                "start_time": "13:00",
                "end_time": "17:00"
            },
            {
                "date": next_friday.strftime("%Y-%m-%d"),
                "location": "fukuoka",
                "start_time": "09:00",
                "end_time": "12:00"
            }
        ]
    })

# [関数] 出張申請メールのテンプレート取得

In [4]:
# ``` GitHub Copilot のPrompt
# 日本語の出張申請のメールのテンプレートを取得します。出張申請の項目として、訪問先、訪問理由、訪問日時を含みます。

def get_business_trip_email_template():
    """Get the business trip email template"""
    print("get_business_trip_email_template called")  
    return json.dumps({
        "email_template": {
            "subject": "出張申請",
            "body": "お疲れ様です。出張の許可をお願いします。以下に出張の詳細を記載します。\n\n訪問先: [訪問先]\n訪問理由: [訪問理由]\n訪問日時: [訪問日時]"
        }
    })


# [関数] 飛行機の予約

In [5]:
# ``` GitHub Copilot のPrompt
# 飛行機の予約を行います。指定した往復の日時と出発地、到着地と、予算の範囲を指定します。予算の範囲の初期値はnullです。
# 予約の結果として、出発地、到着地、出発日時、到着日時、航空会社名、座席クラス、料金を返却します。

def book_flight(departure_date, return_date, departure_location, arrival_location, budget=None):
    """Book a flight with the given departure and return dates, departure and arrival locations, and budget range"""
    print(f"book_flight called with departure_date: {departure_date}, return_date: {return_date}, departure_location: {departure_location}, arrival_location: {arrival_location}, budget: {budget}")  
    return json.dumps({
        "flight_details": {
            "departure_location": departure_location,
            "arrival_location": arrival_location,
            "departure_date": departure_date,
            "return_date": return_date,
            "airline": "JAL",
            "class": "economy",
            "price": "10000"
        }
    })

# [関数] ホテルの予約

In [6]:
# ``` GitHub Copilot のPrompt
# ホテルの予約を行います。チェックイン日、チェックアウト日、ホテル名、部屋数を指定します。
# ホテル名は初期値で「星野リゾート」を指定します

def book_hotel(checkin_date, checkout_date, location, hotel_name="星野リゾート", number_of_rooms=1):
    """Book a hotel with the given check-in and check-out dates, hotel name, and number of rooms"""
    print(f"book_hotel called with checkin_date: {checkin_date}, checkout_date: {checkout_date}, location: {location}, hotel_name: {hotel_name}, number_of_rooms: {number_of_rooms}")  
    return json.dumps({
        "hotel_details": {
            "hotel_name": hotel_name,
            "checkin_date": checkin_date,
            "checkout_date": checkout_date,
            "location": location,
            "number_of_rooms": number_of_rooms,
            "price": "5000"
        }
    })


# [関数] 国内出張規定ドキュメントの取得

In [7]:
# ``` GitHub Copilot のPrompt
# 国内出張規定のドキュメントのファイルの内容を全て取得します。
# ファイルは、 /docs/domestic_travel_policy.txt に保存されているものとします。

def get_domestic_travel_policy():
    """Get the contents of the domestic travel policy document"""
    print("get_domestic_travel_policy called")  
    with open("/docs/domestic_travel_policy.txt", "r") as file:
        return json.dumps({
            "document_content": file.read()
        })


def get_domestic_travel_policy_plaintext():
    with open("docs/domestic_travel_policy.txt", "r") as file:
        return file.read()

# [関数] カレンダーに予定を追加


In [8]:
# カレンダーに予定をを作成します。件名、開始日時、終了日時、場所を指定します。

def create_schedule(subject, start_time, end_time, location):
    """Create a schedule in the calendar with the given subject, start time, end time, and location"""
    print(f"create_schedule called with subject: {subject}, start_time: {start_time}, end_time: {end_time}, location: {location}")  
    return json.dumps({
        "schedule": {
            "subject": subject,
            "start_time": start_time,
            "end_time": end_time,
            "location": location
        }
    })


# [関数] 出張の個人的な好みを取得

In [9]:
# 出張に関してのデフォルト値を取得します。出発地は東京です。出発日時は、スケジュールが午前中の場合は、前日にしてください。宿泊のランクは一般的なビジネス利用でお願いします。交通手段は最短時間を優先します

def get_default_business_trip_values():
    """Get the default values for a business trip"""
    print("get_default_business_trip_values called")  
    return json.dumps({
        "default_values": {
            "departure_location": "tokyo",
            "departure_date": (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d"),
            "hotel_rank": "business",
            "transportation_mode": "shortest"
        }
    })

# Function Calling による Reasoning Engine の呼び出し

## 注意:
- トークン数のカウントをしていません。実際の利用時には、トークン数のカウントを行ってください。
- 例外処理は、適宜追加してください。

In [1]:
# ```Prompt GitHub Copilot Chat
# @workspace Azure OpenAI ServiceのFunction Callingを呼び出すためのtoolsの引数の定義を作成します。選択したファイルに定義されている全ての関数を使います。そして、Function Callingの最終メッセージを作成するための関数呼び出しをハンドルする部分も作成してください。


messages=[
    {
        "role": "system", "content": """
            # 役割:
            あなたは優秀な**旅行代理店**の社員です。お客様の社員の方の出張申請のお手伝いをしてください。

            ## 順守すべきドキュメント:
            出張の規定については、以下の**出張規定のドキュメント**を読み込んで、それを**順守**してください。
            
            """
            +
            get_domestic_travel_policy_plaintext()
            +

            """
            
            ### ユーザーの好み:
            以下がユーザーの好みの情報です。タスクを完了する上で、これらの情報を考慮してください。

            """
            +

            get_default_business_trip_values()

            +
            """
            # タスク:
            ユーザーからの情報に不足がある場合は、ユーザーのカレンダーやユーザーの出張の好みがデフォルト値として定義されている処理を呼び出して、情報を入手してください。それでも情報に不足がある場合は、ユーザーに尋ねてください。
            ユーザーへのメッセージには、必ず出張手配の詳細な進捗状況を含めてください。
            出張手配が完了したら、ユーザーのカレンダーに移動時間と宿泊情報の予定をそれぞれ追加してください。

            深呼吸をして一歩ずつ取り組んでください。
        """
    }
]


def run_conversation():
    user_input = textbox.value    
    messages.append({"role": "user", "content": user_input})

    # Define the functions for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_schedule",
                "description": "来週のスケジュールの情報を取得します",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_business_trip_email_template",
                "description": "出張申請に必要なメールのテンプレートを取得します",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "book_flight",
                "description": "飛行機の予約を行います",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "departure_date": {"type": "string"},
                        "return_date": {"type": "string"},
                        "departure_location": {"type": "string"},
                        "arrival_location": {"type": "string"},
                        "budget": {"type": ["string", "null"]},
                    },
                    "required": ["departure_date", "return_date", "departure_location", "arrival_location"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "book_hotel",
                "description": "ホテルの予約を行います",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "checkin_date": {"type": "string"},
                        "checkout_date": {"type": "string"},
                        "location": {"type": "string"},
                        "hotel_name": {"type": "string", "default": "星野リゾート"},
                        "number_of_rooms": {"type": "integer", "default": 1},
                    },
                    "required": ["checkin_date", "checkout_date", "location"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_domestic_travel_policy",
                "description": "国内出張規定のドキュメントの内容を取得します",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "create_schedule",
                "description": "カレンダーに予定を追加します",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "subject": {"type": "string"},
                        "start_time": {"type": "string"},
                        "end_time": {"type": "string"},
                        "location": {"type": "string"},
                    },
                    "required": ["subject", "start_time", "end_time", "location"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_default_business_trip_values",
                "description": "出張に関する場所や時間の初期値を取得します",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            }
        }
    ]

    # First API call: Ask the model to use the functions
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=max_response_tokens
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Agent Response:" )  
    print(response_message.content)

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function call: {function_name}")  
            print(f"Function arguments: {function_args}")  
            
            if function_name == "get_schedule":
                function_response = get_schedule()
            elif function_name == "get_business_trip_email_template":
                function_response = get_business_trip_email_template()
            elif function_name == "book_flight":
                function_response = book_flight(
                    departure_date=function_args.get("departure_date"),
                    return_date=function_args.get("return_date"),
                    departure_location=function_args.get("departure_location"),
                    arrival_location=function_args.get("arrival_location"),
                    budget=function_args.get("budget")
                )
            elif function_name == "book_hotel":
                function_response = book_hotel(
                    checkin_date=function_args.get("checkin_date"),
                    checkout_date=function_args.get("checkout_date"),
                    location=function_args.get("location"),
                    hotel_name=function_args.get("hotel_name"),
                    number_of_rooms=function_args.get("number_of_rooms")
                )
            elif function_name == "get_domestic_travel_policy":
                function_response = get_domestic_travel_policy()
            elif function_name == "create_schedule":
                function_response = create_schedule(
                    subject=function_args.get("subject"),
                    start_time=function_args.get("start_time"),
                    end_time=function_args.get("end_time"),
                    location=function_args.get("location")
                )
            elif function_name == "get_default_business_trip_values":
                function_response = get_default_business_trip_values()
            else:
                function_response = json.dumps({"error": "Unknown function"})
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
    else:
        print("<system messsage>No tool calls were made by the model.")  

    # Second API call: Get the final response from the model
    final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
    )

    final_response = final_response.choices[0].message.content

    messages.append({"role": "assistant", "content": final_response})

    return final_response

NameError: name 'get_domestic_travel_policy_plaintext' is not defined

# 会話の実行

最初の会話の実行の後、つまり2回目以降は、**このセルの下**に会話結果が表示されます。

In [11]:
import ipywidgets as widgets

textbox = widgets.Text()
button = widgets.Button(description="送信")

def on_click_callback(clicked_button: widgets.Button) -> None:
    print("[Agent Response]\n" + run_conversation())

# ボタンにイベントハンドラを登録する
button.on_click(on_click_callback)

Agent Response:
None
Function call: book_flight
Function arguments: {'departure_date': '2024-08-08', 'return_date': '2024-08-09', 'departure_location': 'tokyo', 'arrival_location': 'fukuoka', 'budget': '20000'}
book_flight called with departure_date: 2024-08-08, return_date: 2024-08-09, departure_location: tokyo, arrival_location: fukuoka, budget: 20000
Function call: book_hotel
Function arguments: {'checkin_date': '2024-08-08', 'checkout_date': '2024-08-09', 'location': 'fukuoka'}
book_hotel called with checkin_date: 2024-08-08, checkout_date: 2024-08-09, location: fukuoka, hotel_name: None, number_of_rooms: None
[Agent Response]
以下の詳細で出張手配が進んでいます。

### 飛行機の予約
- **航空会社**: JAL
- **クラス**: エコノミー
- **出発地**: 東京
- **到着地**: 福岡
- **往路料金**: 10,000円（片道20,000円以内）
- **復路料金**: 10,000円（片道20,000円以内）
- **出発日**: 2024年8月8日
- **帰宅日**: 2024年8月9日

### 宿泊の予約
- **ホテル名**: 現時点で未定
- **チェックイン日**: 2024年8月8日
- **チェックアウト日**: 2024年8月9日
- **宿泊費**: 5,000円（一泊の基準料金内）

### 次のステップ
1. **具体的な飛行機の出発時間帯**（早朝、午前、午後、夜など）を選定します

## 簡易入力の作成

このセルの実行後に会話を続ける場合は、画面に表示されているテキストボックスにPromptを入力して、ボタンを押してください。
入力後の会話の実行は、上のセルの出力部分に数秒後に表示されます。このセルの下では無いので、ご注意ください。

In [12]:
# 入力用の画面を表示し画面
display(textbox)
display(button)

textbox.value = "来週の福岡の出張の手配をしてください。"

# Run the conversation and print the result
print("Agent Response:\n" + run_conversation())

textbox.value = "作業を続けてください。"

Text(value='')

Button(description='送信', style=ButtonStyle())

Agent Response:
None
Function call: get_schedule
Function arguments: {}
get_schedule called
Function call: get_default_business_trip_values
Function arguments: {}
get_default_business_trip_values called
Agent Response:
来週の福岡への出張手配を始めます。以下の情報をもとに手配を進めます。

### 出張日程と内容
- **出張先**: 福岡
- **出発地**: 東京
- **出発日**: 2024年8月8日
- **帰宅日**: 2024年8月9日

### 会議・業務スケジュール
- **8月8日**: 13:00 - 17:00、福岡
- **8月9日**: 09:00 - 12:00、福岡

### 手配内容と進捗状況

#### 1. 飛行機の予約
- **往路**: 東京発 → 福岡行きの飛行機で片道20,000円まで
- **復路**: 福岡発 → 東京行きの飛行機で片道20,000円まで

#### 2. 宿泊の予約
- **ホテル**: ビジネスホテルの基準料金で、1泊15,000円まで

#### 3. その他手配
- **食費**: 1日あたり7,500円まで

現在、以下の情報が不足していますので確認をお願いします。

1. **飛行機の具体的な時間帯**（早朝、午前、午後、夜など）を教えてください。
2. **特別な希望や要望**（座席の連結、追加の手荷物など）について。

以上の情報を確認次第、具体的な手配を進めさせていただきます。また、出張手配が完了したら、カレンダーに移動時間と宿泊情報を追加します。
